In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/FIRE-SMOKE-DATASET/Test'):
    # print(filenames)
    for drn,_,fln in os.walk(dirname):
        print(fln)
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[]
['image_99.jpg', 'image_35.jpg', 'image_0.jpg', 'image_31.jpg', 'image_60.jpg', 'image_59.jpg', 'image_70.jpg', 'image_19.jpg', 'image_28.jpg', 'image_13.jpg', 'image_18.jpg', 'image_32.jpg', 'image_46.jpg', 'image_33.jpg', 'image_30.jpg', 'image_81.jpg', 'image_24.jpg', 'image_37.jpg', 'image_7.jpg', 'image_65.jpg', 'image_79.jpg', 'image_36.jpg', 'image_43.jpg', 'image_87.jpg', 'image_10.jpg', 'image_74.jpg', 'image_54.jpg', 'image_26.jpg', 'image_17.jpg', 'image_63.jpg', 'image_97.jpg', 'image_29.jpg', 'image_78.jpg', 'image_8.jpg', 'image_56.jpg', 'image_6.jpg', 'image_3.jpg', 'image_45.jpg', 'image_11.jpg', 'image_44.jpg', 'image_95.jpg', 'image_50.jpg', 'image_93.jpg', 'image_82.jpg', 'image_42.jpg', 'image_64.jpg', 'image_77.jpg', 'image_69.jpg', 'image_15.jpg', 'image_52.jpg', 'image_2.jpg', 'image_89.jpg', 'image_71.jpg', 'image_57.jpg', 'image_85.jpg', 'image_72.jpg', 'image_58.jpg', 'image_20.jpg', 'image_9.jpg', 'image_91.jpg', 'image_38.jpg', 'image_84.jpg', 'image_5.jp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Load the Pre-trained CNN:
    
    1.1. Import a CNN (e.g., VGG16, ResNet50) from a deep learning library like TensorFlow or PyTorch.

    1.2. Remove the top classification layer of the network to get the feature map output instead. For instance, VGG16 outputs a 4096-dimensional feature vector from its fully connected layers.

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import numpy as np

# Load pre-trained VGG16 model and exclude the final classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


2. Extract Features:

    2.1 Preprocess each image (resize, normalize) to fit the input requirements of the CNN model.
   
    2.2 Pass each image through the feature extractor and flatten the output to create a vector of features.

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

def preprocess_and_extract(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Extract features
    features = feature_extractor.predict(img_array)
    features = features.flatten()
    return features


3: Train the Logistic Regression Model

   3.1 Create a Dataset of Extracted Features:

        
        For each image, extract its feature vector and pair it with the class label (e.g., 1 for fire/smoke and 0 for none).

In [ ]:
import os
img_paths = []
labels = []
#paths = ['Fire','Smoke','Neutral'] # Original code. I suspect you have these inside a single 'Train' folder
paths = ['Train/Fire','Train/Smoke','Train/Neutral'] # Suggest you change it to this

for path in paths:
    PATH = "/content/drive/MyDrive/FIRE-SMOKE-DATASET/"
    #PATH = PATH+path # This is redundant now
    for dirname, _, filenames in os.walk(f'{PATH}/{path}'): # Updated this line to use correct directory
        print(dirname)
        print(len(filenames))
        for filename in filenames:
            img_path = os.path.join(dirname,filename)
            img_paths.append(img_path)
            if 'Fire' in path:  # Updated to match the new paths
                labels.append(1)
            elif 'Smoke' in path:  # Updated to match the new paths
                labels.append(2)
            else:
                labels.append(0)
            # print(os.path.join(dirname, filename))
print(len(img_paths))
print(len(labels))

/content/drive/MyDrive/FIRE-SMOKE-DATASET//Train/Fire
900
/content/drive/MyDrive/FIRE-SMOKE-DATASET//Train/Smoke
900
/content/drive/MyDrive/FIRE-SMOKE-DATASET//Train/Neutral
928
2728
2728


In [ ]:
# Assume img_paths contains paths to images and labels has corresponding labels (1 or 0)
feature_vectors = [preprocess_and_extract(img_path) for img_path in img_paths]
feature_vectors = np.array(feature_vectors)
labels = np.array(labels)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

# Initialize and train logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Evaluate
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.36


In [ ]:
import os
import xml.etree.ElementTree as ET
import numpy as np


def preprocess_and_extract(img_path):
    # Load image (replace with your image loading and feature extraction logic)
    # image = cv2.imread(img_path)
    # ... feature extraction ...
    # For this example, assume a random feature vector is generated
    return np.random.rand(10)

img_paths = []
labels = []
#paths = ['Fire','Smoke','Neutral'] # Original code. I suspect you have these inside a single 'Train' folder
paths = ['Train/Fire','Train/Smoke','Train/Neutral'] # Suggest you change it to this

for path in paths:
    PATH = "/content/drive/MyDrive/FIRE-SMOKE-DATASET/"
    #PATH = PATH+path # This is redundant now
    # The problem was this line: os.walk(f'{PATH}/{path}')
    # It was creating incorrect paths like '/content/drive/MyDrive/FIRE-SMOKE-DATASET//content/drive/MyDrive/FIRE-SMOKE-DATASET/Train/Fire'
    # Change it to the following to use correct path:
    for dirname, _, filenames in os.walk(os.path.join(PATH, path)):
        print(dirname)
        print(len(filenames))
        for filename in filenames:
            img_path = os.path.join(dirname,filename)
            img_paths.append(img_path)
            if 'Fire' in path:  # Updated to match the new paths
                labels.append(1)
            elif 'Smoke' in path:  # Updated to match the new paths
                labels.append(2)
            else:
                labels.append(0)
            # print(os.path.join(dirname, filename))

print(len(img_paths))
print(len(labels))

feature_vectors = [preprocess_and_extract(img_path) for img_path in img_paths]
feature_vectors = np.array(feature_vectors)
labels = np.array(labels)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

# Initialize and train logistic regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Evaluate
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

/content/drive/MyDrive/FIRE-SMOKE-DATASET/Train/Fire
900
/content/drive/MyDrive/FIRE-SMOKE-DATASET/Train/Smoke
900
/content/drive/MyDrive/FIRE-SMOKE-DATASET/Train/Neutral
928
2728
2728
Model Accuracy: 0.34


3.2 Train the Logistic Regression Model:

    With the extracted feature vectors, train a logistic regression model on this new dataset.

In [ ]:
print(f"Number of image paths: {len(img_paths)}")
print(f"Number of labels: {len(labels)}")




Number of image paths: 2728
Number of labels: 122


Step 4: Make Predictions

With the trained logistic regression model, you can now make predictions on new images. Just preprocess and extract features as before, then pass these features to the logistic regression model.

In [ ]:
def predict_fire_smoke(img_path):
    features = preprocess_and_extract(img_path)
    prediction = log_reg.predict([features])
    # return "Fire/Smoke Detected" if prediction[0] == 1 else "No Fire/Smoke Detected"
    if prediction == 1:
        return "Fire"
    elif prediction == 2:
        return "Smoke"
    elif prediction == 0:
        return "Neutral"

# Example usage
print(predict_fire_smoke("/content/drive/MyDrive/FIRE-SMOKE-DATASET/Test/Fire/image_0.jpg"))
print(predict_fire_smoke("/content/drive/MyDrive/FIRE-SMOKE-DATASET/Test/Neutral/image_0.jpg"))
print(predict_fire_smoke("/content/drive/MyDrive/FIRE-SMOKE-DATASET/Test/Smoke/image_0.jpg"))

Fire
Smoke
Smoke


##### why did i try the above method
 -->Summary -->

    The pre-trained CNN extracts meaningful features from each image.

    The logistic regression model, trained on these features, acts as a classifier to detect fire or smoke.
        
    This approach leverages the powerful feature extraction capabilities of CNNs while keeping the classification model lightweight with logistic regression.